In [ ]:
## Python basics
import os
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import random
from IPython.core.pylabtools import figsize
from time import time
from scipy.stats import norm, uniform
import math
import collections
import collections.abc
from collections import OrderedDict    # For recording the model specification 
collections.Iterable = collections.abc.Iterable
import pylogit as pl  
import lxml

## DNN
import tensorflow as tf
import tensorflow_lattice as tfl
from scikeras.wrappers import KerasRegressor, KerasClassifier
from tensorflow.keras.layers import Activation,Input, Dense, Reshape, Concatenate, Layer, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, Embedding, Flatten,LeakyReLU,ReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import RMSprop, Adam

from sklearn.metrics import accuracy_score,f1_score
from statsmodels.formula.api import logit
import statsmodels.api as sm
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.multiclass import unique_labels
import torch.nn.functional as F
import torch
from torch import tensor
import torch.distributions.log_normal as log_normal
import torch.distributions.bernoulli as bernoulli 


## PDP
from sklearn.inspection import PartialDependenceDisplay, partial_dependence
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error,mean_absolute_percentage_error,accuracy_score,mean_squared_error
from tensorflow.keras import regularizers
from sklearn.metrics import brier_score_loss


# Get the prediction metrics
def brier_multi(targets, probs):
     return np.mean(np.sum((probs - targets)**2, axis=1))
def save_clipboard(X):
    pd.DataFrame(X).to_clipboard(index=False,header=False)
np.set_printoptions(suppress=True)

pd.options.mode.chained_assignment = None  # default='warn'

def compute_VOT_POP(X,Y):
    VOT_temp = []
    for i in range(X.shape[0]):
        for j in range(Y.shape[0]):
            VOT_temp.append(X[i]/Y[j])
    return np.array(VOT_temp)

def compute_VOT_IND(X,Y):
    VOT_temp = []
    for i in range(X.shape[0]):
        #VOT_temp.append(compute_VOT_POP(X[i,:],Y[i,:]).mean())
         VOT_temp.append(np.median(compute_VOT_POP(X[i,:],Y[i,:])))
    return np.array(VOT_temp)

import warnings

#suppress warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
import tensorflow as tf

# gpu = tf.config.experimental.get_visible_devices('GPU')[0]
# tf.config.experimental.set_memory_growth(device = gpu, enable = True)

In [ ]:
df_wide = pd.read_csv("data/data_METRO.csv")
df_wide = df_wide[(df_wide.AGE != 6) & (df_wide.CHOICE != 0) & (df_wide.INCOME != 4) & (df_wide.PURPOSE != 9) & (df_wide.CAR_AV != 0)]

# Create AGE dummy
df_wide["AGES"] = 0
df_wide.loc[df_wide["AGE"] == 1,"AGES"] = 1
df_wide.loc[df_wide["AGE"] == 2,"AGES"] = 2
df_wide.loc[df_wide["AGE"] == 3,"AGES"] = 3
df_wide.loc[df_wide["AGE"] == 4,"AGES"] = 4
df_wide.loc[df_wide["AGE"] == 5,"AGES"] = 5

# Create INCOME dummy
df_wide["INCOMES"] = 0
df_wide.loc[df_wide["INCOME"].isin([0,1]),"INCOMES"] = 1
df_wide.loc[df_wide["INCOME"] == 2,"INCOMES"] = 2
df_wide.loc[df_wide["INCOME"] == 3,"INCOMES"] = 3
df_wide.loc[df_wide["INCOME"] == 4,"INCOMES"] = 4


# Create PURPOSE dummy
df_wide["PURPOSES"] = 0
df_wide.loc[df_wide["PURPOSE"].isin([1,5]),"PURPOSES"] = 1
df_wide.loc[df_wide["PURPOSE"].isin([2,6]),"PURPOSES"] = 2
df_wide.loc[df_wide["PURPOSE"].isin([3,7]),"PURPOSES"] = 3
df_wide.loc[df_wide["PURPOSE"].isin([4,8]),"PURPOSES"] = 4



# Scale the travel time column by 60 to convert raw units (minutes) to hours
df_wide["TRAIN_TT"] = df_wide["TRAIN_TT"] / 100.0
df_wide["SM_TT"] = df_wide["SM_TT"] / 100.0
df_wide["CAR_TT"] = df_wide["CAR_TT"] / 100.0

# Scale the headway column by 60 to convert raw units (minutes) to hours
df_wide["TRAIN_HE"] = df_wide["TRAIN_HE"] / 100.0
df_wide["SM_HE"] = df_wide["SM_HE"] / 100.0

df_wide["TRAIN_TC"] = df_wide["TRAIN_CO"] / 100.0
df_wide["SM_TC"] = df_wide["SM_CO"] / 100.0
df_wide["CAR_TC"] = df_wide["CAR_CO"] / 100.0

df_wide.loc[(df_wide["GA"] == 1),"TRAIN_TC"] = 0
df_wide.loc[(df_wide["GA"] == 1),"SM_TC"] = 0


##########
# Create various dummy variables to describe the choice context of a given
# invidual for each choice task.
##########

# Create AGE dummy
#df_wide["AGE_1"] = (df_wide["AGE"] == 1).astype(int)
df_wide["AGE_2"] = (df_wide["AGE"] == 2).astype(int)
df_wide["AGE_3"] = (df_wide["AGE"] == 3).astype(int)
df_wide["AGE_4"] = (df_wide["AGE"] == 4).astype(int)
df_wide["AGE_5"] = (df_wide["AGE"] == 5).astype(int)

# Create INCOME dummy
#df_wide["INCOME_01"] = (df_wide["INCOME"].isin([0,1])).astype(int)
df_wide["INCOME_2"] = (df_wide["INCOME"] == 2).astype(int)
df_wide["INCOME_3"] = (df_wide["INCOME"] == 3).astype(int)
df_wide["INCOME_4"] = (df_wide["INCOME"] == 4).astype(int)

# Create PURPOSE dummy
#df_wide["PURPOSE_1"] = (df_wide["PURPOSE"].isin([1,5])).astype(int)
df_wide["PURPOSE_2"] = (df_wide["PURPOSE"].isin([2,6])).astype(int)
df_wide["PURPOSE_3"] = (df_wide["PURPOSE"].isin([3,7])).astype(int)
df_wide["PURPOSE_4"] = (df_wide["PURPOSE"].isin([4,8])).astype(int)

# Create LUGGAGE dummy
#df_wide["LUGGAGE_0"] = (df_wide["LUGGAGE"] == 0).astype(int)
df_wide["LUGGAGE_1"] = (df_wide["LUGGAGE"] == 1).astype(int)
df_wide["LUGGAGE_3"] = (df_wide["LUGGAGE"] == 3).astype(int)

# Create a dummy variable indicating that a person is NOT first class
df_wide["TRAIN_CLASS"] = 1 - df_wide["FIRST"]


In [ ]:
df_wide['TRAIN_TT']

In [ ]:
# Create the list of individual specific variables
ind_variables = ['ID','GA','AGE_2', 'AGE_3', 'AGE_4',
       'AGE_5', 'INCOME_2', 'INCOME_3', 'INCOME_4', 'PURPOSE_2', 'PURPOSE_3',
       'PURPOSE_4','AGES', 'INCOMES','PURPOSES']


# Specify the variables that vary across individuals and some or all alternatives
# The keys are the column names that will be used in the long format dataframe.
# The values are dictionaries whose key-value pairs are the alternative id and
# the column name of the corresponding column that encodes that variable for
# the given alternative. Examples below.
alt_varying_variables = {u'TT': dict([(1, 'TRAIN_TT'),
                                               (2, 'SM_TT'),
                                               (3, 'CAR_TT')]),
                          u'TC': dict([(1, 'TRAIN_TC'),
                                                (2, 'SM_TC'),
                                                (3, 'CAR_TC')]),
                          u'HE': dict([(1, 'TRAIN_HE'),
                                            (2, 'SM_HE')]),
                          u'SEAT': dict([(2, "SM_SEATS")])}

# Specify the availability variables
# Note that the keys of the dictionary are the alternative id's.
# The values are the columns denoting the availability for the
# given mode in the dataset.
availability_variables = {1: 'TRAIN_AV',
                          2: 'SM_AV', 
                          3: 'CAR_AV'}

##########
# Determine the columns for: alternative ids, the observation ids and the choice
##########
# The 'custom_alt_id' is the name of a column to be created in the long-format data
# It will identify the alternative associated with each row.
custom_alt_id = "mode_id"

# Create a custom id column that ignores the fact that this is a 
# panel/repeated-observations dataset. Note the +1 ensures the id's start at one.
obs_id_column = "custom_id"
df_wide[obs_id_column] = np.arange(df_wide.shape[0], dtype=int) + 1

# Create a variable recording the choice column
choice_column = "CHOICE"

In [ ]:
# Perform the conversion to long-format
df_long = pl.convert_wide_to_long(df_wide, 
                                           ind_variables, 
                                           alt_varying_variables, 
                                           availability_variables, 
                                           obs_id_column, 
                                           choice_column,
                                           new_alt_id_name=custom_alt_id)

In [ ]:
df_long["TTxAGE_2"] = df_long["TT"]*df_long["AGE_2"]
df_long["TTxAGE_3"] = df_long["TT"]*df_long["AGE_3"]
df_long["TTxAGE_4"] = df_long["TT"]*df_long["AGE_4"]
df_long["TTxAGE_5"] = df_long["TT"]*df_long["AGE_5"]

df_long["TTxINCOME_2"] = df_long["TT"]*df_long["INCOME_2"]
df_long["TTxINCOME_3"] = df_long["TT"]*df_long["INCOME_3"]

df_long["TTxPURPOSE_2"] = df_long["TT"]*df_long["PURPOSE_2"]
df_long["TTxPURPOSE_3"] = df_long["TT"]*df_long["PURPOSE_3"]
df_long["TTxPURPOSE_4"] = df_long["TT"]*df_long["PURPOSE_4"]

## 1. MNL-A

In [ ]:
# Alternative-specific Utility Specification

basic_specification = OrderedDict()
basic_names = OrderedDict()

basic_specification["intercept"] = [1, 2]
basic_names["intercept"] = ['ASC Train',
                            'ASC Swissmetro']

basic_specification["TT"] = [1, 2, 3]
basic_names["TT"] = ['Travel Time(Train)',
                     'Travel Time(Swissmetro)',
                     'Travel Time(Car)']

basic_specification["TC"] = [[1, 2, 3]]
basic_names["TC"] = ['Travel Cost(All)']

basic_specification["HE"] = [1, 2]
basic_names["HE"] = ["Headway(Train)",
                     "Headway(Swissmetro)"]

basic_specification["SEAT"] = [2]
basic_names["SEAT"] = ['Airline Seat(Swissmetro)']

basic_specification["GA"] = [1,2]
basic_names["GA"] = ['GA(Train)',
                     'GA(Swissmetro)']


In [ ]:
from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import train_test_split

#Original: 1005

SS = GroupShuffleSplit(n_splits=2, train_size=0.7, random_state=1234)
train_idx, rem_idx = next(SS.split(df_wide,groups = df_wide['custom_id']))

df_wide_train = df_wide.iloc[train_idx]
df_wide_rem = df_wide.iloc[rem_idx]

SS2 = GroupShuffleSplit(n_splits=2, train_size=0.5, random_state=1234)
test_idx, valid_idx = next(SS2.split(df_wide_rem,groups = df_wide_rem['custom_id']))
df_wide_test = df_wide_rem.iloc[test_idx]
df_wide_valid = df_wide_rem.iloc[valid_idx]

df_train = df_long[df_long.custom_id.isin(df_wide_train['custom_id'].unique())]
df_test = df_long[df_long.custom_id.isin(df_wide_test['custom_id'].unique())]



# Estimate the multinomial logit model (MNL)
MNL_A = pl.create_choice_model(data=df_train,
                                alt_id_col=custom_alt_id,
                                obs_id_col=obs_id_column,
                                choice_col=choice_column,
                                specification=basic_specification,
                                model_type="MNL",
                                names=basic_names)

# Specify the initial values and method for the optimization.
MNL_A.fit_mle(np.zeros(11),maxiter=3000)


# Get the prediction metrics
predict_train = np.argmax(MNL_A.predict(df_train).reshape(-1,3),axis=1)
predict_test =  np.argmax(MNL_A.predict(df_test).reshape(-1,3),axis=1)

y_train = np.array(df_train.CHOICE).reshape(-1,3)
y_test = np.array(df_test.CHOICE).reshape(-1,3)
y_train_cat = np.argmax(np.array(df_train.CHOICE).reshape(-1,3),axis=1)
y_test_cat = np.argmax(np.array(df_test.CHOICE).reshape(-1,3),axis=1)


train_p = MNL_A.predict(df_train).reshape(-1,3)
test_p = MNL_A.predict(df_test).reshape(-1,3)

test_NLL = round((F.cross_entropy(input=torch.log(tensor(test_p)),target=torch.Tensor(y_test)).numpy()).item(),3)
train_NLL =  round((F.cross_entropy(input=torch.log(tensor(train_p)),target=torch.Tensor(y_train)).numpy()).item(),3)


train_acc = round(accuracy_score(y_train_cat,predict_train),3)
test_acc = round(accuracy_score(y_test_cat,predict_test),3)

train_brier = round(brier_multi(y_train,train_p),3)
test_brier = round(brier_multi(y_test,test_p),3)

print([train_acc,test_acc,train_NLL,test_NLL,train_brier,test_brier])
pd.DataFrame(np.array([train_acc,test_acc,train_NLL,test_NLL,train_brier,test_brier])).transpose().to_clipboard(index=False,header=False)
# Look at the estimation results
MNL_A.get_statsmodels_summary()

In [ ]:
# Get the utility
param_table = MNL_A.get_statsmodels_summary().tables[1].as_html()
param_table = pd.read_html(param_table, header=0, index_col=0)[0]
param_coef = param_table['coef']
df_TR = df_long.loc[df_long['mode_id'] == 1,['TT','TC','HE','GA']]
df_SM = df_long.loc[df_long['mode_id'] == 2,['TT','TC','HE','GA','SEAT']]
df_CAR = df_long.loc[df_long['mode_id'] == 3,['TT','TC']]

util_TR = param_table['coef']['ASC Train'] + np.dot(np.array(df_TR),np.array(param_table['coef'][['Travel Time(Train)','Travel Cost(All)','Headway(Train)','GA(Train)']]))
util_SM = param_table['coef']['ASC Swissmetro'] + np.dot(np.array(df_SM),np.array(param_table['coef'][['Travel Time(Swissmetro)','Travel Cost(All)','Headway(Swissmetro)','GA(Swissmetro)','Airline Seat(Swissmetro)']]))
util_CAR = np.dot(np.array(df_CAR),np.array(param_table['coef'][['Travel Time(Car)','Travel Cost(All)']]))

print(util_TR.min(),util_TR.max(),util_SM.min(),util_SM.max(),util_CAR.min(),util_CAR.max())

In [ ]:
# Get the individual-level VOT 
def first(x):
    return(x.iloc[1,:])

VOT_TR_A =  param_coef['Travel Time(Train)']/param_coef['Travel Cost(All)']
VOT_SM_A =  param_coef['Travel Time(Swissmetro)']/param_coef['Travel Cost(All)']
VOT_CAR_A =  param_coef['Travel Time(Car)']/param_coef['Travel Cost(All)']

print(np.array([VOT_TR_A,VOT_SM_A,VOT_CAR_A,]).round(3))

## 2. MNL-B

In [ ]:
# Alternative-specific Utility Specification

basic_specification = OrderedDict()
basic_names = OrderedDict()

basic_specification["intercept"] = [1, 2]
basic_names["intercept"] = ['ASC Train',
                            'ASC Swissmetro']

basic_specification["TT"] = [1, 2, 3]
basic_names["TT"] = ['Travel Time(Train)',
                     'Travel Time(Swissmetro)',
                     'Travel Time(Car)']

basic_specification["TC"] = [[1, 2, 3]]
basic_names["TC"] = ['Travel Cost(All)']
                     

basic_specification["HE"] = [1, 2]
basic_names["HE"] = ["Headway(Train)",
                     "Headway(Swissmetro)"]

basic_specification["SEAT"] = [2]
basic_names["SEAT"] = ['Airline Seat(Swissmetro)']

basic_specification["GA"] = [1,2]
basic_names["GA"] = ['GA(Train)',
                     'GA(Swissmetro)']

## First-order Interactions
basic_specification["TTxAGE_2"] = [1, 2, 3]
basic_names["TTxAGE_2"] = ["TTxAGE_2(Train)",
                           "TTxAGE_2(Swissmetro)",
                           "TTxAGE_2(Car)"]

basic_specification["TTxAGE_3"] = [1, 2, 3]
basic_names["TTxAGE_3"] = ["TTxAGE_3(Train)",
                           "TTxAGE_3(Swissmetro)",
                           "TTxAGE_3(Car)"]

basic_specification["TTxAGE_4"] = [1, 2, 3]
basic_names["TTxAGE_4"] = ["TTxAGE_4(Train)",
                           "TTxAGE_4(Swissmetro)",
                           "TTxAGE_4(Car)"]

basic_specification["TTxAGE_5"] = [1, 2, 3]
basic_names["TTxAGE_5"] = ["TTxAGE_5(Train)",
                           "TTxAGE_5(Swissmetro)",
                           "TTxAGE_5(Car)"]

basic_specification["TTxINCOME_2"] = [1, 2, 3]
basic_names["TTxINCOME_2"] = ["TTxINCOME_2(Train)",
                              "TTxINCOME_2(Swissmetro)",
                              "TTxINCOME_2(Car)"]

basic_specification["TTxINCOME_3"] = [1, 2, 3]
basic_names["TTxINCOME_3"] = ["TTxINCOME_3(Train)",
                              "TTxINCOME_3(Swissmetro)",
                              "TTxINCOME_3(Car)"]

basic_specification["TTxPURPOSE_2"] = [1, 2, 3]
basic_names["TTxPURPOSE_2"] = ["TTxPURPOSE_2(Train)",
                              "TTxPURPOSE_2(Swissmetro)",
                              "TTxPURPOSE_2(Car)"]

basic_specification["TTxPURPOSE_3"] = [1, 2, 3]
basic_names["TTxPURPOSE_3"] = ["TTxPURPOSE_3(Train)",
                              "TTxPURPOSE_3(Swissmetro)",
                              "TTxPURPOSE_3(Car)"]

basic_specification["TTxPURPOSE_4"] = [1, 2, 3]
basic_names["TTxPURPOSE_4"] = ["TTxPURPOSE_4(Train)",
                              "TTxPURPOSE_4(Swissmetro)",
                              "TTxPURPOSE_4(Car)"]

In [ ]:
from sklearn.model_selection import GroupShuffleSplit


# Estimate the multinomial logit model (MNL)
MNL_B = pl.create_choice_model(data=df_train,
                                alt_id_col=custom_alt_id,
                                obs_id_col=obs_id_column,
                                choice_col=choice_column,
                                specification=basic_specification,
                                model_type="MNL",
                                names=basic_names)

# Specify the initial values and method for the optimization.
MNL_B.fit_mle(np.zeros(38))

# Look at the estimation results
MNL_B.get_statsmodels_summary()

In [ ]:
# Get the prediction metrics
predict_train = np.argmax(MNL_B.predict(df_train).reshape(-1,3),axis=1)
predict_test =  np.argmax(MNL_B.predict(df_test).reshape(-1,3),axis=1)

y_train = np.array(df_train.CHOICE).reshape(-1,3)
y_test = np.array(df_test.CHOICE).reshape(-1,3)
y_train_cat = np.argmax(np.array(df_train.CHOICE).reshape(-1,3),axis=1)
y_test_cat = np.argmax(np.array(df_test.CHOICE).reshape(-1,3),axis=1)


train_p = MNL_B.predict(df_train).reshape(-1,3)
test_p = MNL_B.predict(df_test).reshape(-1,3)

test_NLL = round((F.cross_entropy(input=torch.log(tensor(test_p)),target=torch.Tensor(y_test)).numpy()).item(),3)
train_NLL =  round((F.cross_entropy(input=torch.log(tensor(train_p)),target=torch.Tensor(y_train)).numpy()).item(),3)

train_acc = round(accuracy_score(y_train_cat,predict_train),3)
test_acc = round(accuracy_score(y_test_cat,predict_test),3)

train_brier = round(brier_multi(y_train,train_p),3)
test_brier = round(brier_multi(y_test,test_p),3)

print([train_acc,test_acc,train_NLL,test_NLL,train_brier,test_brier])
pd.DataFrame(np.array([train_acc,test_acc,train_NLL,test_NLL,train_brier,test_brier])).transpose().to_clipboard(index=False)

In [ ]:
# Get the individual-level VOT 
def first(x):
    return(x.iloc[1,:])

param_table = MNL_B.get_statsmodels_summary().tables[1].as_html()
param_table = pd.read_html(param_table, header=0, index_col=0)[0]
param_coef = param_table['coef']
df_VOT_B = df_test[['ID','custom_id','AGE_2', 'AGE_3','AGE_4', 'AGE_5',
                  'INCOME_2', 'INCOME_3', 'PURPOSE_2','PURPOSE_3', 'PURPOSE_4','AGES','INCOMES','PURPOSES']].groupby('custom_id').apply(first)
df_VOT_B['intercept'] = np.ones(df_VOT_B.shape[0])
VOT_TR_B = np.dot(
        np.array(df_VOT_B[['intercept','AGE_2', 'AGE_3','AGE_4', 'AGE_5',
                'INCOME_2', 'INCOME_3', 'PURPOSE_2','PURPOSE_3', 'PURPOSE_4']]),
        np.array(param_coef[['Travel Time(Train)',
                'TTxAGE_2(Train)','TTxAGE_3(Train)','TTxAGE_4(Train)','TTxAGE_5(Train)','TTxINCOME_2(Train)','TTxINCOME_3(Train)',
                'TTxPURPOSE_2(Train)','TTxPURPOSE_3(Train)','TTxPURPOSE_4(Train)']]))

VOT_SM_B = np.dot(
        np.array(df_VOT_B[['intercept','AGE_2', 'AGE_3','AGE_4','AGE_5',
                'INCOME_2', 'INCOME_3', 'PURPOSE_2','PURPOSE_3','PURPOSE_4']]),
        np.array(param_coef[['Travel Time(Swissmetro)',
                'TTxAGE_2(Swissmetro)','TTxAGE_3(Swissmetro)','TTxAGE_4(Swissmetro)','TTxAGE_5(Swissmetro)','TTxINCOME_2(Swissmetro)','TTxINCOME_3(Swissmetro)',
                'TTxPURPOSE_2(Swissmetro)','TTxPURPOSE_3(Swissmetro)','TTxPURPOSE_4(Swissmetro)']]))

VOT_CAR_B = np.dot(
        np.array(df_VOT_B[['intercept','AGE_2', 'AGE_3','AGE_4', 'AGE_5',
                'INCOME_2', 'INCOME_3', 'PURPOSE_2','PURPOSE_3', 'PURPOSE_4']]),
        np.array(param_coef[['Travel Time(Car)',
                'TTxAGE_2(Car)','TTxAGE_3(Car)','TTxAGE_4(Car)','TTxAGE_5(Car)','TTxINCOME_2(Car)','TTxINCOME_3(Car)',
                'TTxPURPOSE_2(Car)','TTxPURPOSE_3(Car)','TTxPURPOSE_4(Car)']]))

df_VOT_B['VOT_TR'] = VOT_TR_B / param_coef['Travel Cost(All)']
df_VOT_B['VOT_SM'] = VOT_SM_B / param_coef['Travel Cost(All)']
df_VOT_B['VOT_CAR'] = VOT_CAR_B / param_coef['Travel Cost(All)']

df_VOT_B = df_VOT_B[['ID','VOT_TR','VOT_SM','VOT_CAR',
                     'AGES','INCOMES','PURPOSES']].groupby(['AGES','INCOMES','PURPOSES']).mean()

print(np.quantile(df_VOT_B['VOT_TR'],(0.5,0.01,0.25,0.75,0.99)).round(3))
print(np.quantile(df_VOT_B['VOT_SM'],(0.5,0.01,0.25,0.75,0.99)).round(3))
print(np.quantile(df_VOT_B['VOT_CAR'],(0.5,0.01,0.25,0.75,0.99)).round(3))


## 3. DNN

In [ ]:
## 1. DNN

x_train = df_wide_train[[
    'TRAIN_TT','TRAIN_TC','TRAIN_HE',
    'SM_TT','SM_TC','SM_HE','SM_SEATS',
    'CAR_TT', 'CAR_TC','GA','AGES','INCOMES','PURPOSES']]

y_train = np.array(pd.get_dummies(df_wide_train['CHOICE']))


x_test = df_wide_test[[
    'TRAIN_TT','TRAIN_TC','TRAIN_HE',
    'SM_TT','SM_TC','SM_HE','SM_SEATS',
    'CAR_TT', 'CAR_TC','GA','AGES','INCOMES','PURPOSES']]

y_test = np.array(pd.get_dummies(df_wide_test['CHOICE']))

x_valid = df_wide_valid[[
    'TRAIN_TT','TRAIN_TC','TRAIN_HE',
    'SM_TT','SM_TC','SM_HE','SM_SEATS',
    'CAR_TT', 'CAR_TC','GA','AGES','INCOMES','PURPOSES']]

y_valid = np.array(pd.get_dummies(df_wide_valid['CHOICE']))

In [ ]:
## 2. Define the Original DNN

def build_DNN(num_layers,num_neurons,drop_rate,learning_rate):

    img = Input(shape=x_train.shape[1],name="main_input")   
    h = Dense(num_neurons,activation='relu')(img)
    h = Dropout(drop_rate)(h)
    for num_layer in range(num_layers-1):
        h = Dense(num_neurons,activation='relu')(h)
        h = Dropout(drop_rate)(h)
    util = Dense(1)(h)
    
    ## Alternative-specific utility
    util_ALT1 = Dense(int(num_neurons*0.5))(h)
    util_ALT1 = Dropout(drop_rate)(util_ALT1)
    util_ALT1 = Dense(1,name='output_TR')(util_ALT1)
    
    util_ALT2 = Dense(int(num_neurons*0.5))(h)
    util_ALT2 = Dropout(drop_rate)(util_ALT2)
    util_ALT2 = Dense(1,name='output_SM')(util_ALT2)
       
    util_ALT3 = Dense(int(num_neurons*0.5))(h)
    util_ALT3 = Dropout(drop_rate)(util_ALT3)
    util_ALT3 = Dense(1,name='output_CAR')(util_ALT3)

    out_prob =  tf.keras.layers.Softmax(name='out_prob')(Concatenate()([util_ALT1,util_ALT2,util_ALT3]))
    
    model = Model(img,[out_prob,util_ALT1,util_ALT2,util_ALT3])  
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=['categorical_crossentropy',None,None,None],
                  metrics=['accuracy',None,None,None])

    return model

In [ ]:
for i in range(0,1):
    
    num_layers = 3
    num_neurons = 200
    drop_rate = 0.005
    learning_rate = 0.0005

    DNN = build_DNN(num_layers,num_neurons,drop_rate,learning_rate)

    batch_size = 128
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', restore_best_weights=True, verbose=1, patience=20)
    history = DNN.fit(
        x=x_train,
        y=y_train,
        shuffle=True,
        epochs = 200,
        batch_size = batch_size,
        validation_data=[x_valid,y_valid],
        callbacks = [es],
        verbose=0)

    DNN.save_weights('C:/Users/euijin/Documents/DNN_Models_Revision/DNN_FUL_'+str(i))
    #DNN.load_weights('C:/Users/euijin/Documents/DNN_Models/DNN_FUL_'+str(i))



    # Get the prediction metrics
    test_p = DNN.predict(x_test,verbose=0)
    train_p = DNN.predict(x_train,verbose=0)

    predict_train = np.argmax(train_p[0],axis=1)
    predict_test =  np.argmax(test_p[0],axis=1)

    y_train = np.array(df_train.CHOICE).reshape(-1,3)
    y_test = np.array(df_test.CHOICE).reshape(-1,3)
    y_train_cat = np.argmax(np.array(df_train.CHOICE).reshape(-1,3),axis=1)
    y_test_cat = np.argmax(np.array(df_test.CHOICE).reshape(-1,3),axis=1)

    train_acc = round(accuracy_score(y_train_cat,predict_train),3)
    test_acc = round(accuracy_score(y_test_cat,predict_test),3)

    train_brier = round(brier_multi(y_train,train_p[0]),3)
    test_brier = round(brier_multi(y_test,test_p[0]),3)

    print([train_acc,test_acc,train_brier,test_brier])


    ## Plot the PD and ICE
    from sklearn.inspection import partial_dependence
    from sklearn.inspection import PartialDependenceDisplay
    import matplotlib.pyplot as plt

    class DNNFunction(BaseEstimator, ClassifierMixin):
        def fit(self, X, y):
            self.classes_ = unique_labels(y)
            return self

        def predict_proba(self, X):
            self.proba_,_,_,_ = DNN.predict(X,verbose=0)
            return np.array(self.proba_)

        def decision_function(self, X):
            self.utility_ = np.array(DNN.predict(X)[1:4]).transpose().reshape(-1,3)
            return self.utility_

    DNN_ALL = DNNFunction()
    DNN_ALL.fit(x_train,y_train)

    common_params = {
        "subsample": 0.99999,
        "n_jobs": 1,
        "grid_resolution": 20,
        "centered": True,
        "random_state": 1,
        "response_method": 'decision_function',
        "percentiles": (0.01, 0.99),

    }

    fig,ax = plt.subplots(nrows=1,ncols=2, figsize=(12,6),sharex=False,sharey=True)

    display_0 = PartialDependenceDisplay.from_estimator(
        DNN_ALL,
        x_train,
        features=['TRAIN_TT','TRAIN_TC'], # TC,TT
        kind=["both","both"],
        ice_lines_kw={"color": "gray",'alpha':0.3},pd_line_kw={"color": "black","lw":3,'linestyle':'solid','label':'True'},
        ax=ax,
        target = 0,
        **common_params,
    )

    fig,ax = plt.subplots(nrows=1,ncols=2, figsize=(12,6),sharex=False,sharey=True)

    display_1 = PartialDependenceDisplay.from_estimator(
        DNN_ALL,
        x_train,
        features=['SM_TT','SM_TC'], # TC,TT
        kind=["both","both"],
        ice_lines_kw={"color": "gray",'alpha':0.3},pd_line_kw={"color": "black","lw":3,'linestyle':'solid','label':'True'},
        ax=ax,
        target = 1,
        **common_params,
    )

    fig,ax = plt.subplots(nrows=1,ncols=2, figsize=(12,6),sharex=False,sharey=True)

    display_2 = PartialDependenceDisplay.from_estimator(
        DNN_ALL,
        x_train,
        features=['CAR_TT','CAR_TC'], # TC,TT
        kind=["both","both"],
        ice_lines_kw={"color": "gray",'alpha':0.3},pd_line_kw={"color": "black","lw":3,'linestyle':'solid','label':'True'},
        ax=ax,
        target = 2,
        **common_params,
    )



    ## Population-level Parameters

    PD_DNN_TT_TR = display_0.pd_results[0]['average'][0]
    PD_DNN_TC_TR = display_0.pd_results[1]['average'][0] 

    PD_DNN_TT_SM = display_1.pd_results[0]['average'][1]
    PD_DNN_TC_SM = display_1.pd_results[1]['average'][1] 

    PD_DNN_TT_CAR = display_2.pd_results[0]['average'][2]
    PD_DNN_TC_CAR = display_2.pd_results[1]['average'][2] 

    PD_values_TT_TR = display_0.pd_results[0]['values'][0]
    PD_values_TC_TR = display_0.pd_results[1]['values'][0]

    PD_values_TT_SM = display_1.pd_results[0]['values'][0]
    PD_values_TC_SM = display_1.pd_results[1]['values'][0]

    PD_values_TT_CAR = display_2.pd_results[0]['values'][0]
    PD_values_TC_CAR = display_2.pd_results[1]['values'][0]

    beta_DNN_TT_TR = (np.diff(PD_DNN_TT_TR)/np.diff(PD_values_TT_TR))
    beta_DNN_TC_TR = (np.diff(PD_DNN_TC_TR)/np.diff(PD_values_TC_TR))

    beta_DNN_TT_SM = (np.diff(PD_DNN_TT_SM)/np.diff(PD_values_TT_SM))
    beta_DNN_TC_SM = (np.diff(PD_DNN_TC_SM)/np.diff(PD_values_TC_SM))

    beta_DNN_TT_CAR = (np.diff(PD_DNN_TT_CAR)/np.diff(PD_values_TT_CAR))
    beta_DNN_TC_CAR = (np.diff(PD_DNN_TC_CAR)/np.diff(PD_values_TC_CAR))


    TC_TR_IDX =(beta_DNN_TC_TR != 0)
    TC_SM_IDX =(beta_DNN_TC_SM != 0)
    TC_CAR_IDX =(beta_DNN_TC_CAR != 0)

    beta_DNN_TC_TR = beta_DNN_TC_TR[TC_TR_IDX]
    beta_DNN_TC_SM = beta_DNN_TC_SM[TC_SM_IDX]
    beta_DNN_TC_CAR = beta_DNN_TC_CAR[TC_CAR_IDX]

    beta_DNN_TT_TR = beta_DNN_TT_TR[TC_TR_IDX]
    beta_DNN_TT_SM = beta_DNN_TT_SM[TC_SM_IDX]
    beta_DNN_TT_CAR = beta_DNN_TT_CAR[TC_CAR_IDX]


    beta_DNN_VOT_POP_TR = np.median(compute_VOT_POP(beta_DNN_TT_TR,beta_DNN_TC_TR))
    beta_DNN_VOT_POP_SM = np.median(compute_VOT_POP(beta_DNN_TT_SM,beta_DNN_TC_SM))
    beta_DNN_VOT_POP_CAR = np.median(compute_VOT_POP(beta_DNN_TT_CAR,beta_DNN_TC_CAR))


    ## Individual-level Parameters

    ICE_DNN_TT_TR = display_0.pd_results[0]['individual'][0]
    ICE_DNN_TC_TR = display_0.pd_results[1]['individual'][0]

    ICE_DNN_TT_SM = display_1.pd_results[0]['individual'][1]
    ICE_DNN_TC_SM = display_1.pd_results[1]['individual'][1]

    ICE_DNN_TT_CAR = display_2.pd_results[0]['individual'][2]
    ICE_DNN_TC_CAR = display_2.pd_results[1]['individual'][2]

    beta_DNN_TT_TR_ICE = (np.diff(ICE_DNN_TT_TR)/np.diff(PD_values_TT_TR))
    beta_DNN_TC_TR_ICE = (np.diff(ICE_DNN_TC_TR)/np.diff(PD_values_TC_TR))
    beta_DNN_VOT_TR_ICE = compute_VOT_IND(beta_DNN_TT_TR_ICE[:,TC_TR_IDX],beta_DNN_TC_TR_ICE[:,TC_TR_IDX])

    beta_DNN_TT_SM_ICE = (np.diff(ICE_DNN_TT_SM)/np.diff(PD_values_TT_SM))
    beta_DNN_TC_SM_ICE = (np.diff(ICE_DNN_TC_SM)/np.diff(PD_values_TC_SM))
    beta_DNN_VOT_SM_ICE = compute_VOT_IND(beta_DNN_TT_SM_ICE[:,TC_SM_IDX],beta_DNN_TC_SM_ICE[:,TC_SM_IDX])


    beta_DNN_TT_CAR_ICE = (np.diff(ICE_DNN_TT_CAR)/np.diff(PD_values_TT_CAR))
    beta_DNN_TC_CAR_ICE = (np.diff(ICE_DNN_TC_CAR)/np.diff(PD_values_TC_CAR))
    beta_DNN_VOT_CAR_ICE = compute_VOT_IND(beta_DNN_TT_CAR_ICE[:,TC_CAR_IDX],beta_DNN_TC_CAR_ICE[:,TC_CAR_IDX])


    # Estimation # If we use the median value, we don't need to care about the outlier or somethings.

    x_test_E = x_train[['AGES','INCOMES','PURPOSES']].reset_index(drop=True)
    x_test_E['VOT_TR'] = beta_DNN_VOT_TR_ICE
    x_test_E['VOT_SM'] = beta_DNN_VOT_SM_ICE
    x_test_E['VOT_CAR'] = beta_DNN_VOT_CAR_ICE

    VOT_DNN_IND = x_test_E[['AGES','INCOMES','PURPOSES','VOT_TR','VOT_SM','VOT_CAR']].groupby(['AGES','INCOMES','PURPOSES']).median()


    print(np.quantile(VOT_DNN_IND['VOT_TR'],(0.5,0.01,0.25,0.75,0.99)).round(3))
    print(np.quantile(VOT_DNN_IND['VOT_SM'],(0.5,0.01,0.25,0.75,0.99)).round(3))
    print(np.quantile(VOT_DNN_IND['VOT_CAR'],(0.5,0.01,0.25,0.75,0.99)).round(3))


    # Estimation results
    PRED_PERF = np.array([train_acc,test_acc,train_brier,test_brier]).transpose()
    EST_RESULTS = [PRED_PERF,VOT_DNN_IND]
    with open("C:/Users/euijin/Documents/DNN_Results_Revision/FUL_RESULTS_FF_"+str(i), "wb") as f:
        pickle.dump(EST_RESULTS, f)
    #save_clipboard(np.concatenate([PRED_PERF,VOT_DNN_IND.median(axis=0).round(3)])) 

In [ ]:
import scipy.stats as stats
## Results analysis
PRED_PERF_S = []
VOT_DNN_IND_S = []
VOT_DNN_index = []

for i in range(1):
    with open("C:/Users/euijin/Documents/DNN_Results_Revision/FUL_RESULTS_FF_"+str(i),'rb') as f:
        temp = pickle.load(f)
    PRED_PERF_S.append(temp[0])
    VOT_DNN_IND_S.append(temp[1])
    VOT_DNN_index.append(temp[1].index.values)
    

## Predictability    
PRED_PERF_S = np.array(PRED_PERF_S)
A = np.concatenate([PRED_PERF_S.mean(axis=0).round(3).reshape(-1,1),PRED_PERF_S.std(axis=0).round(3).reshape(-1,1)],axis=1)


## Interpretability
IND_stats = []
VOT_DNN_IND_S_TR = []
VOT_DNN_IND_S_SM = []
VOT_DNN_IND_S_CAR = []

for i in range(len(VOT_DNN_IND_S)):
    
    inter_IDX_IND = VOT_DNN_IND_S[i].index.isin(np.intersect1d(VOT_DNN_IND_S[i].index,df_VOT_B.index))
        
    TRIND = np.quantile(VOT_DNN_IND_S[i].iloc[inter_IDX_IND,0],(0.5,0.01,0.25,0.75,0.99)).round(3)
    SMIND = np.quantile(VOT_DNN_IND_S[i].iloc[inter_IDX_IND,1],(0.5,0.01,0.25,0.75,0.99)).round(3)
    CARIND = np.quantile(VOT_DNN_IND_S[i].iloc[inter_IDX_IND,2],(0.5,0.01,0.25,0.75,0.99)).round(3)   
    ALLIND = np.concatenate([TRIND,SMIND,CARIND])   
    IND_stats.append(ALLIND)
    
    VOT_DNN_IND_S_TR.append(VOT_DNN_IND_S[i].iloc[inter_IDX_IND,0])
    VOT_DNN_IND_S_SM.append(VOT_DNN_IND_S[i].iloc[inter_IDX_IND,1])
    VOT_DNN_IND_S_CAR.append(VOT_DNN_IND_S[i].iloc[inter_IDX_IND,2])
      
B = np.array(IND_stats)
B = np.concatenate([np.median(B,axis=0).round(3).reshape(-1,1),
                    stats.iqr(B,axis=0).round(3).reshape(-1,1)],axis=1)


VOT_DNN_IND_S_TR = np.concatenate(VOT_DNN_IND_S_TR)
VOT_DNN_IND_S_SM = np.concatenate(VOT_DNN_IND_S_SM)
VOT_DNN_IND_S_CAR = np.concatenate(VOT_DNN_IND_S_CAR)

EST_RESULTS_DNN = np.vstack([B,A])
save_clipboard(EST_RESULTS_DNN)

## 4. TCNN

In [ ]:
## 2. BCDNN

x_train = df_wide_train[[
    'TRAIN_TT','TRAIN_TC','TRAIN_HE',
    'SM_TT','SM_TC','SM_HE','SM_SEATS',
    'CAR_TT', 'CAR_TC','GA','AGES','INCOMES','PURPOSES']]

y_train = np.array(pd.get_dummies(df_wide_train['CHOICE']))


x_test = df_wide_test[[
    'TRAIN_TT','TRAIN_TC','TRAIN_HE',
    'SM_TT','SM_TC','SM_HE','SM_SEATS',
    'CAR_TT', 'CAR_TC','GA','AGES','INCOMES','PURPOSES']]

y_test = np.array(pd.get_dummies(df_wide_test['CHOICE']))

x_valid = df_wide_valid[[
    'TRAIN_TT','TRAIN_TC','TRAIN_HE',
    'SM_TT','SM_TC','SM_HE','SM_SEATS',
    'CAR_TT', 'CAR_TC','GA','AGES','INCOMES','PURPOSES']]

y_valid = np.array(pd.get_dummies(df_wide_valid['CHOICE']))

In [ ]:
## Hyperparameters of Lattice Networks

def build_LatDNN(TRAIN_TT_KP,TRAIN_TT_LS,TRAIN_TC_KP,TRAIN_TC_LS,TRAIN_HE_KP,TRAIN_HE_LS,SM_TT_KP,SM_TT_LS,SM_TC_KP,SM_TC_LS,SM_HE_KP,SM_HE_LS,
                 CAR_TT_KP,CAR_TT_LS,CAR_TC_KP,CAR_TC_LS,SM_SEATS_LS,GA_LS,AGE_LS,INCOME_LS,PURPOSE_LS,ACT_TR,ACT_SM,ACT_CAR):
    
    non_split_input = Input(shape=x_train.shape[1], name='non_split_input',dtype='float32')
    
    # Alternative-specific Lattice inputs
    lattice_inputs_TR = []
    lattice_inputs_SM = []
    lattice_inputs_CAR = []
    
    # TRAIN_TT
    TRAIN_TT_input = tf.reshape(non_split_input[:,0],(-1,1))
    
    ## TRAIN_TT to TR
    TRAIN_TT_TR_calibrator = tfl.layers.PWLCalibration(
    input_keypoints=np.linspace(
        x_train['TRAIN_TT'].min(), x_train['TRAIN_TT'].max(), num=TRAIN_TT_KP),
    dtype=tf.float32,
    output_min=0.0,
    output_max=TRAIN_TT_LS - 1.0,
    kernel_regularizer=[("wrinkle", 0.0, 0.5),('hessian', 0.0, 0.0001)],
    monotonicity='decreasing', ## Sign constraints 
    name='TRAIN_TT_TR_Calib',
    )(TRAIN_TT_input)

    ## TRAIN_TT to Others
    TRAIN_TT_OTH_calibrator = tfl.layers.PWLCalibration(
    input_keypoints=np.linspace(
        x_train['TRAIN_TT'].min(), x_train['TRAIN_TT'].max(), num=TRAIN_TT_KP),
    dtype=tf.float32,
    output_min=0.0,
    output_max=TRAIN_TT_LS - 1.0,
    kernel_regularizer=[("wrinkle", 0.0, 0.5),('hessian', 0.0, 0.0001)],
    monotonicity='increasing',
    name='TRAIN_TT_OTH_Calib',
    )(TRAIN_TT_input)
   

    lattice_inputs_TR.append(TRAIN_TT_TR_calibrator)
    # lattice_inputs_SM.append(TRAIN_TT_OTH_calibrator)
    # lattice_inputs_CAR.append(TRAIN_TT_OTH_calibrator)      
    
    
    # TRAIN_TC
    TRAIN_TC_input = tf.reshape(non_split_input[:,1],(-1,1))
    
    ## TRAIN_TC to TR
    TRAIN_TC_TR_calibrator = tfl.layers.PWLCalibration(
    input_keypoints=np.linspace(
        x_train['TRAIN_TC'].min(), x_train['TRAIN_TC'].max(), num=TRAIN_TC_KP),
    dtype=tf.float32,
    output_min=0.0,
    output_max=TRAIN_TC_LS - 1.0,
    kernel_regularizer=[("wrinkle", 0.0, 0.5),('hessian', 0.0, 0.0001)],
    monotonicity='decreasing', ## Sign constraints 
    name='TRAIN_TC_TR_Calib',
    )(TRAIN_TC_input)


    ## TRAIN_TC to Others
    TRAIN_TC_OTH_calibrator = tfl.layers.PWLCalibration(
    input_keypoints=np.linspace(
        x_train['TRAIN_TC'].min(), x_train['TRAIN_TC'].max(), num=TRAIN_TC_KP),
    dtype=tf.float32,
    output_min=0.0,
    output_max=TRAIN_TC_LS - 1.0,
    kernel_regularizer=[("wrinkle", 0.0, 0.5),('hessian', 0.0, 0.0001)],
    monotonicity='increasing',
    name='TRAIN_TC_OTH_Calib',
    )(TRAIN_TC_input)
    
    lattice_inputs_TR.append(TRAIN_TC_TR_calibrator)
    # lattice_inputs_SM.append(TRAIN_TC_OTH_calibrator)
    # lattice_inputs_CAR.append(TRAIN_TC_OTH_calibrator)
    
    
        
    # TRAIN_HE
    TRAIN_HE_input = tf.reshape(non_split_input[:,2],(-1,1))
    
    ## TRAIN_HE to TR
    TRAIN_HE_TR_calibrator = tfl.layers.PWLCalibration(
    input_keypoints=np.linspace(
        x_train['TRAIN_HE'].min(), x_train['TRAIN_HE'].max(), num=TRAIN_HE_KP),
    dtype=tf.float32,
    output_min=0.0,
    output_max=TRAIN_HE_LS - 1.0,
    kernel_regularizer=[("wrinkle", 0.0, 0.5),('hessian', 0.0, 0.0001)],
    monotonicity='decreasing', ## Sign constraints 
    name='TRAIN_HE_TR_Calib',
    )(TRAIN_HE_input)


    ## TRAIN_HE to Others
    TRAIN_HE_OTH_calibrator = tfl.layers.PWLCalibration(
    input_keypoints=np.linspace(
        x_train['TRAIN_HE'].min(), x_train['TRAIN_HE'].max(), num=TRAIN_HE_KP),
    dtype=tf.float32,
    output_min=0.0,
    output_max=TRAIN_HE_LS - 1.0,
    kernel_regularizer=[("wrinkle", 0.0, 0.5),('hessian', 0.0, 0.0001)],
    monotonicity='increasing', ## Sign constraints 
    name='TRAIN_HE_OTH_Calib',
    )(TRAIN_HE_input)
   
    lattice_inputs_TR.append(TRAIN_HE_TR_calibrator)
    # lattice_inputs_SM.append(TRAIN_HE_OTH_calibrator)
    # lattice_inputs_CAR.append(TRAIN_HE_OTH_calibrator)
    
    
    
    
    # Swissmetro_TT
    SM_TT_input = tf.reshape(non_split_input[:,3],(-1,1))
    
    ## SM_TT to SM
    SM_TT_SM_calibrator = tfl.layers.PWLCalibration(
    input_keypoints=np.linspace(
        x_train['SM_TT'].min(), x_train['SM_TT'].max(), num=SM_TT_KP),
    dtype=tf.float32,
    output_min=0.0,
    output_max=SM_TT_LS - 1.0,
    kernel_regularizer=[("wrinkle", 0.0, 0.5),('hessian', 0.0, 0.0001)],
    monotonicity='decreasing', ## Sign constraints 
    name='SM_TT_SM_Calib',
    )(SM_TT_input)

    ## SM_TT to Others
    SM_TT_OTH_calibrator = tfl.layers.PWLCalibration(
    input_keypoints=np.linspace(
        x_train['SM_TT'].min(), x_train['SM_TT'].max(), num=SM_TT_KP),
    dtype=tf.float32,
    output_min=0.0,
    output_max=SM_TT_LS - 1.0,
    kernel_regularizer=[("wrinkle", 0.0, 0.5),('hessian', 0.0, 0.0001)],
    monotonicity='increasing', ## Sign constraints 
    name='SM_TT_OTH_Calib',
    )(SM_TT_input)
    lattice_inputs_SM.append(SM_TT_SM_calibrator)
    # lattice_inputs_TR.append(SM_TT_OTH_calibrator)
    # lattice_inputs_CAR.append(SM_TT_OTH_calibrator)
    
    
    # Swissmetro_TC
    SM_TC_input = tf.reshape(non_split_input[:,4],(-1,1))
    
    ## SM_TC to SM
    SM_TC_SM_calibrator = tfl.layers.PWLCalibration(
    input_keypoints=np.linspace(
        x_train['SM_TC'].min(), x_train['SM_TC'].max(), num=SM_TC_KP),
    dtype=tf.float32,
    output_min=0.0,
    output_max=SM_TC_LS - 1.0,
    kernel_regularizer=[("wrinkle", 0.0, 0.5),('hessian', 0.0, 0.0001)],
    monotonicity='decreasing', ## Sign constraints 
    name='SM_TC_SM_Calib',
    )(SM_TC_input)

    ## SM_TC to Others
    SM_TC_OTH_calibrator = tfl.layers.PWLCalibration(
    input_keypoints=np.linspace(
        x_train['SM_TC'].min(), x_train['SM_TC'].max(), num=SM_TC_KP),
    dtype=tf.float32,
    output_min=0.0,
    output_max=SM_TC_LS - 1.0,
    kernel_regularizer=[("wrinkle", 0.0, 0.5),('hessian', 0.0, 0.0001)],
    monotonicity='increasing', ## Sign constraints 
    name='SM_TC_OTH_Calib',
    )(SM_TC_input)

    lattice_inputs_SM.append(SM_TC_SM_calibrator)
    # lattice_inputs_TR.append(SM_TC_OTH_calibrator)
    # lattice_inputs_CAR.append(SM_TC_OTH_calibrator)
    
  

    # Swissmetro_HE
    SM_HE_input = tf.reshape(non_split_input[:,5],(-1,1))
    
    ## SM_HE to SM
    SM_HE_SM_calibrator = tfl.layers.PWLCalibration(
    input_keypoints=np.linspace(
        x_train['SM_HE'].min(), x_train['SM_HE'].max(), num=SM_HE_KP),
    dtype=tf.float32,
    output_min=0.0,
    output_max=SM_HE_LS - 1.0,
    kernel_regularizer=[("wrinkle", 0.0, 0.5),('hessian', 0.0, 0.0001)],
    monotonicity='decreasing', ## Sign constraints 
    name='SM_HE_SM_Calib',
    )(SM_HE_input)

    ## SM_HE to Others
    SM_HE_OTH_calibrator = tfl.layers.PWLCalibration(
    input_keypoints=np.linspace(
        x_train['SM_HE'].min(), x_train['SM_HE'].max(), num=SM_HE_KP),
    dtype=tf.float32,
    output_min=0.0,
    output_max=SM_HE_LS - 1.0,
    kernel_regularizer=[("wrinkle", 0.0, 0.5),('hessian', 0.0, 0.0001)],
    monotonicity='increasing', ## Sign constraints 
    name='SM_HE_OTH_Calib',
    )(SM_HE_input)

    lattice_inputs_SM.append(SM_HE_SM_calibrator)
    # lattice_inputs_TR.append(SM_HE_OTH_calibrator)
    # lattice_inputs_CAR.append(SM_HE_OTH_calibrator)
    
    
    
    
    # CAR_TT
    CAR_TT_input = tf.reshape(non_split_input[:,7],(-1,1))
    
    ## CAR_TT to CAR
    CAR_TT_CAR_calibrator = tfl.layers.PWLCalibration(
    input_keypoints=np.linspace(
        x_train['CAR_TT'].min(), x_train['CAR_TT'].max(), num=CAR_TT_KP),
    dtype=tf.float32,
    output_min=0.0,
    output_max=CAR_TT_LS - 1.0,
    kernel_regularizer=[("wrinkle", 0.0, 0.5),('hessian', 0.0, 0.0001)],
    monotonicity='decreasing', ## Sign constraints 
    name='CAR_TT_CAR_Calib',
    )(CAR_TT_input)

    ## CAR_TT to Others
    CAR_TT_OTH_calibrator = tfl.layers.PWLCalibration(
    input_keypoints=np.linspace(
        x_train['CAR_TT'].min(), x_train['CAR_TT'].max(), num=CAR_TT_KP),
    dtype=tf.float32,
    output_min=0.0,
    output_max=CAR_TT_LS - 1.0,
    kernel_regularizer=[("wrinkle", 0.0, 0.5),('hessian', 0.0, 0.0001)],
    monotonicity='increasing', ## Sign constraints 
    name='CAR_TT_OTH_Calib',
    )(CAR_TT_input)
    lattice_inputs_CAR.append(CAR_TT_CAR_calibrator)
    # lattice_inputs_TR.append(CAR_TT_OTH_calibrator)
    # lattice_inputs_SM.append(CAR_TT_OTH_calibrator)
    
    
    # CAR_TC
    CAR_TC_input = tf.reshape(non_split_input[:,8],(-1,1))
    
    ## CAR_TC to CAR
    CAR_TC_CAR_calibrator = tfl.layers.PWLCalibration(
    input_keypoints=np.linspace(
        x_train['CAR_TC'].min(), x_train['CAR_TC'].max(), num=CAR_TC_KP),
    dtype=tf.float32,
    output_min=0.0,
    output_max=CAR_TC_LS - 1.0,
    kernel_regularizer=[("wrinkle", 0.0, 0.5),('hessian', 0.0, 0.0001)],
    monotonicity='decreasing', ## Sign constraints 
    name='CAR_TC_CAR_Calib',
    )(CAR_TC_input)

    ## CAR_TC to Others
    CAR_TC_OTH_calibrator = tfl.layers.PWLCalibration(
    input_keypoints=np.linspace(
        x_train['CAR_TC'].min(), x_train['CAR_TC'].max(), num=CAR_TC_KP),
    dtype=tf.float32,
    output_min=0.0,
    output_max=CAR_TC_LS - 1.0,
    kernel_regularizer=[("wrinkle", 0.0, 0.5),('hessian', 0.0, 0.0001)],
    monotonicity='increasing', ## Sign constraints 
    name='CAR_TC_OTH_Calib',
    )(CAR_TC_input)
    lattice_inputs_CAR.append(CAR_TC_CAR_calibrator)
    # lattice_inputs_TR.append(CAR_TC_OTH_calibrator)
    # lattice_inputs_SM.append(CAR_TC_OTH_calibrator)
       
       
    
    
    ## Non-monotonic attributes
    
    ## SM_SEATS
    SM_SEATS_input = tf.reshape(non_split_input[:,6],(-1,1))
    SM_SEATS_calibrator = tfl.layers.CategoricalCalibration(
        num_buckets=2,
        output_min=0.0,
        output_max=SM_SEATS_LS - 1.0,
        name='SM_SEATS_calib',
    )(SM_SEATS_input)
    lattice_inputs_CAR.append(SM_SEATS_calibrator)
    lattice_inputs_TR.append(SM_SEATS_calibrator)
    lattice_inputs_SM.append(SM_SEATS_calibrator)

    ## GA
    GA_input = tf.reshape(non_split_input[:,9],(-1,1))
    GA_calibrator = tfl.layers.CategoricalCalibration(
        num_buckets=2,
        output_min=0.0,
        output_max=GA_LS - 1.0,
        name='GA_SM_calib',
    )(GA_input)
    lattice_inputs_CAR.append(GA_calibrator)
    lattice_inputs_TR.append(GA_calibrator)
    lattice_inputs_SM.append(GA_calibrator)
    
    ## AGE
    AGE_input = tf.reshape(non_split_input[:,10],(-1,1))
    AGE_calibrator = tfl.layers.CategoricalCalibration(
        num_buckets=5,
        output_min=0.0,
        output_max=AGE_LS - 1.0,
        name='AGE_SM_calib',
    )(AGE_input)
    lattice_inputs_CAR.append(AGE_calibrator)
    lattice_inputs_TR.append(AGE_calibrator)
    lattice_inputs_SM.append(AGE_calibrator)

    ## INCOME
    INCOME_input = tf.reshape(non_split_input[:,11],(-1,1))
    INCOME_calibrator = tfl.layers.CategoricalCalibration(
        num_buckets=3,
        output_min=0.0,
        output_max=INCOME_LS - 1.0,
        name='INCOME_calib',
    )(INCOME_input)
    lattice_inputs_CAR.append(INCOME_calibrator)
    lattice_inputs_TR.append(INCOME_calibrator)
    lattice_inputs_SM.append(INCOME_calibrator)
    
    
    ## PURPOSE
    PURPOSE_input = tf.reshape(non_split_input[:,12],(-1,1))
    PURPOSE_calibrator = tfl.layers.CategoricalCalibration(
        num_buckets=4,
        output_min=0.0,
        output_max=PURPOSE_LS - 1.0,
        name='PURPOSE_calib',
    )(PURPOSE_input)
    lattice_inputs_CAR.append(PURPOSE_calibrator)
    lattice_inputs_TR.append(PURPOSE_calibrator)
    lattice_inputs_SM.append(PURPOSE_calibrator)   
    

    ### Non-lienarly fuse the outputs of calibrator
    util_ALT1 = tfl.layers.Lattice(   
    lattice_sizes=[TRAIN_TT_LS,TRAIN_TC_LS,TRAIN_HE_LS,
                   SM_SEATS_LS,GA_LS,AGE_LS,INCOME_LS,PURPOSE_LS],
    monotonicities=[
        'increasing', 'increasing','increasing',
        'none','none','none','none','none'],

    output_min=0,
    output_max=1,
    # output_min=-100,
    # output_max=100,
    name='lattice_TR',
    )(lattice_inputs_TR)
    
    
    ### Non-lienarly fuse the outputs of calibrator
    util_ALT2 = tfl.layers.Lattice(   
    lattice_sizes=[SM_TT_LS,SM_TC_LS,SM_HE_LS,
                   SM_SEATS_LS,GA_LS,AGE_LS,INCOME_LS,PURPOSE_LS],
    monotonicities=[
        'increasing', 'increasing','increasing',
        'none','none','none','none','none'],

    output_min=0,
    output_max=1,
    # output_min=-100,
    # output_max=100,
    name='lattice_SM',
    )(lattice_inputs_SM)
    
    ## Non-lienarly fuse the outputs of calibrator
    util_ALT3= tfl.layers.Lattice(   
    lattice_sizes=[CAR_TT_LS,CAR_TC_LS,
                   SM_SEATS_LS,GA_LS,AGE_LS,INCOME_LS,PURPOSE_LS],
    monotonicities=[
        'increasing', 'increasing',
        'none','none','none','none','none'],

    output_min=0,
    output_max=1,
    # output_min=-100,
    # output_max=100,
    name='lattice_CAR',
    )(lattice_inputs_CAR)
    
    ## Output PWLCalibrator is the key of improving predictability
    util_ALT1 = tfl.layers.PWLCalibration(
    input_keypoints=np.linspace(0.0, 1.0, ACT_TR),
    output_min=-100,
    output_max=100,
    name='output_TR')(util_ALT1)
  
    util_ALT2 = tfl.layers.PWLCalibration(
    input_keypoints=np.linspace(0.0, 1.0, ACT_SM),
    output_min=-100,
    output_max=100,
    name='output_SM')(util_ALT2)
    
    
    util_ALT3 = tfl.layers.PWLCalibration(
    input_keypoints=np.linspace(0.0, 1.0, ACT_CAR),
    output_min=-100,
    output_max=100,
    name='output_CAR')(util_ALT3)

    out_prob =  tf.keras.layers.Softmax(name='out_prob')(Concatenate()([util_ALT1,util_ALT2,util_ALT3]))
    
    model = Model(non_split_input,[out_prob,util_ALT1,util_ALT2,util_ALT3])   

    optimizer = Adam(learning_rate=learning_rate)
           
    model.compile(optimizer=optimizer, loss=['categorical_crossentropy',None,None,None],
                  metrics=['accuracy',None,None,None])
    
    return model

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_absolute_percentage_error
import pickle
import time

## Repeated Experiments
for i in range(50):

    # record start time
    start = time.time()

    TRAIN_TT_KP = 10
    SM_TT_KP = 30
    CAR_TT_KP = 10

    TRAIN_TC_KP = 20 # SNU2
    SM_TC_KP = 10 # SNU2
    CAR_TC_KP = 10
    TRAIN_HE_KP = 10
    SM_HE_KP = 30

    TRAIN_TT_LS = 4
    SM_TT_LS = 4
    CAR_TT_LS = 4

    TRAIN_TC_LS = 4 #
    SM_TC_LS = 2
    CAR_TC_LS = 4

    TRAIN_HE_LS = 2   
    SM_HE_LS = 2
    SM_SEATS_LS =2
    
    GA_LS = 2
    AGE_LS = 3
    INCOME_LS = 2
    PURPOSE_LS = 4

    ACT_TR = 2
    ACT_SM = 2
    ACT_CAR = 2
    learning_rate = 0.005 #0.0001 is very stable / 0.0005 is not BAD   
    
    LatDNN = build_LatDNN(TRAIN_TT_KP,TRAIN_TT_LS,TRAIN_TC_KP,TRAIN_TC_LS,TRAIN_HE_KP,TRAIN_HE_LS,SM_TT_KP,SM_TT_LS,SM_TC_KP,SM_TC_LS,SM_HE_KP,SM_HE_LS,
                     CAR_TT_KP,CAR_TT_LS,CAR_TC_KP,CAR_TC_LS,SM_SEATS_LS,GA_LS,AGE_LS,INCOME_LS,PURPOSE_LS,ACT_TR,ACT_SM,ACT_CAR)

    batch_size = 128
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', restore_best_weights=True, verbose=1, patience=20)

    history = LatDNN.fit(
        x=x_train,
        y=y_train,
        shuffle=True,
        epochs =200,
        batch_size = batch_size,
        validation_data = [x_valid,y_valid],
        callbacks = [es],
        verbose=0)
    

    LatDNN.save_weights('C:/Users/euijin/Documents/LatDNN_Models_Revision/LatDNN_ASC_FF_'+str(i))
    #LatDNN.load_weights('C:/Users/euijin/Documents/LatDNN_Models/LatDNN_ASC_FF_SNU2_'+str(i))
    
    # record end time
    end = time.time()
    training_time = (end-start)


    # Get the prediction metrics
    test_p = LatDNN.predict(x_test,verbose=0)
    train_p = LatDNN.predict(x_train,verbose=0)

    predict_train = np.argmax(train_p[0],axis=1)
    predict_test =  np.argmax(test_p[0],axis=1)

    y_train = np.array(df_train.CHOICE).reshape(-1,3)
    y_test = np.array(df_test.CHOICE).reshape(-1,3)
    y_train_cat = np.argmax(np.array(df_train.CHOICE).reshape(-1,3),axis=1)
    y_test_cat = np.argmax(np.array(df_test.CHOICE).reshape(-1,3),axis=1)

    test_NLL = round((F.cross_entropy(input=torch.log(tensor(test_p[0])),target=torch.Tensor(y_test)).numpy()).item(),3)
    train_NLL =  round((F.cross_entropy(input=torch.log(tensor(train_p[0])),target=torch.Tensor(y_train)).numpy()).item(),3)

    train_acc = round(accuracy_score(y_train_cat,predict_train),3)
    test_acc = round(accuracy_score(y_test_cat,predict_test),3)

    train_brier = round(brier_multi(y_train,train_p[0]),3)
    test_brier = round(brier_multi(y_test,test_p[0]),3)

    print([train_acc,test_acc,train_NLL,test_NLL,train_brier,test_brier])

    ## Plot the PD and ICE
    from sklearn.inspection import partial_dependence
    from sklearn.inspection import PartialDependenceDisplay
    import matplotlib.pyplot as plt

    class LatDNNFunction(BaseEstimator, ClassifierMixin):
        def fit(self, X, y):
            self.classes_ = unique_labels(y)
            return self

        def predict_proba(self, X):
            self.proba_,_,_,_ = LatDNN.predict(X,verbose=0)
            return np.array(self.proba_)

        def decision_function(self, X):
            self.utility_ = np.array(LatDNN.predict(X)[1:4]).transpose().reshape(-1,3)
            return self.utility_

    LatDNN_ALL = LatDNNFunction()
    LatDNN_ALL.fit(x_train,y_train)

    common_params = {
        "subsample": 0.99999,
        "n_jobs": 1,
        "grid_resolution": 20,
        "centered": True,
        "random_state": 1,
        "response_method": 'decision_function',
        "percentiles": (0.01, 0.99),

    }

    fig,ax = plt.subplots(nrows=1,ncols=2, figsize=(12,6),sharex=False,sharey=True)

    display_0 = PartialDependenceDisplay.from_estimator(
        LatDNN_ALL,
        x_train,
        features=['TRAIN_TT','TRAIN_TC'], # TC,TT
        kind=["both","both"],
        ice_lines_kw={"color": "gray",'alpha':0.3},pd_line_kw={"color": "black","lw":3,'linestyle':'solid','label':'True'},
        ax=ax,
        target = 0,
        **common_params,
    )

    fig,ax = plt.subplots(nrows=1,ncols=2, figsize=(12,6),sharex=False,sharey=True)

    display_1 = PartialDependenceDisplay.from_estimator(
        LatDNN_ALL,
        x_train,
        features=['SM_TT','SM_TC'], # TC,TT
        kind=["both","both"],
        ice_lines_kw={"color": "gray",'alpha':0.3},pd_line_kw={"color": "black","lw":3,'linestyle':'solid','label':'True'},
        ax=ax,
        target = 1,
        **common_params,
    )

    fig,ax = plt.subplots(nrows=1,ncols=2, figsize=(12,6),sharex=False,sharey=True)
    
    display_2 = PartialDependenceDisplay.from_estimator(
        LatDNN_ALL,
        x_train,
        features=['CAR_TT','CAR_TC'], # TC,TT
        kind=["both","both"],
        ice_lines_kw={"color": "gray",'alpha':0.3},pd_line_kw={"color": "black","lw":3,'linestyle':'solid','label':'True'},
        ax=ax,
        target = 2,
        **common_params,
    )



    ## Population-level Parameters

    PD_TCNN_TT_TR = display_0.pd_results[0]['average'][0]
    PD_TCNN_TC_TR = display_0.pd_results[1]['average'][0] 

    PD_TCNN_TT_SM = display_1.pd_results[0]['average'][1]
    PD_TCNN_TC_SM = display_1.pd_results[1]['average'][1] 

    PD_TCNN_TT_CAR = display_2.pd_results[0]['average'][2]
    PD_TCNN_TC_CAR = display_2.pd_results[1]['average'][2] 

    PD_values_TT_TR = display_0.pd_results[0]['values'][0]
    PD_values_TC_TR = display_0.pd_results[1]['values'][0]
    
    PD_values_TT_SM = display_1.pd_results[0]['values'][0]
    PD_values_TC_SM = display_1.pd_results[1]['values'][0]
    
    PD_values_TT_CAR = display_2.pd_results[0]['values'][0]
    PD_values_TC_CAR = display_2.pd_results[1]['values'][0]

    beta_TCNN_TT_TR = (np.diff(PD_TCNN_TT_TR)/np.diff(PD_values_TT_TR))
    beta_TCNN_TC_TR = (np.diff(PD_TCNN_TC_TR)/np.diff(PD_values_TC_TR))

    beta_TCNN_TT_SM = (np.diff(PD_TCNN_TT_SM)/np.diff(PD_values_TT_SM))
    beta_TCNN_TC_SM = (np.diff(PD_TCNN_TC_SM)/np.diff(PD_values_TC_SM))

    beta_TCNN_TT_CAR = (np.diff(PD_TCNN_TT_CAR)/np.diff(PD_values_TT_CAR))
    beta_TCNN_TC_CAR = (np.diff(PD_TCNN_TC_CAR)/np.diff(PD_values_TC_CAR))


    TC_TR_IDX =(beta_TCNN_TC_TR != 0)
    TC_SM_IDX =(beta_TCNN_TC_SM != 0)
    TC_CAR_IDX =(beta_TCNN_TC_CAR != 0)

    beta_TCNN_TC_TR = beta_TCNN_TC_TR[TC_TR_IDX]
    beta_TCNN_TC_SM = beta_TCNN_TC_SM[TC_SM_IDX]
    beta_TCNN_TC_CAR = beta_TCNN_TC_CAR[TC_CAR_IDX]
    
    beta_TCNN_TT_TR = beta_TCNN_TT_TR[TC_TR_IDX]
    beta_TCNN_TT_SM = beta_TCNN_TT_SM[TC_SM_IDX]
    beta_TCNN_TT_CAR = beta_TCNN_TT_CAR[TC_CAR_IDX]

    
    beta_TCNN_VOT_POP_TR = np.median(compute_VOT_POP(beta_TCNN_TT_TR,beta_TCNN_TC_TR))
    beta_TCNN_VOT_POP_SM = np.median(compute_VOT_POP(beta_TCNN_TT_SM,beta_TCNN_TC_SM))
    beta_TCNN_VOT_POP_CAR = np.median(compute_VOT_POP(beta_TCNN_TT_CAR,beta_TCNN_TC_CAR))


    ## Individual-level Parameters

    ICE_TCNN_TT_TR = display_0.pd_results[0]['individual'][0]
    ICE_TCNN_TC_TR = display_0.pd_results[1]['individual'][0]

    ICE_TCNN_TT_SM = display_1.pd_results[0]['individual'][1]
    ICE_TCNN_TC_SM = display_1.pd_results[1]['individual'][1]

    ICE_TCNN_TT_CAR = display_2.pd_results[0]['individual'][2]
    ICE_TCNN_TC_CAR = display_2.pd_results[1]['individual'][2]

    beta_TCNN_TT_TR_ICE = (np.diff(ICE_TCNN_TT_TR)/np.diff(PD_values_TT_TR))
    beta_TCNN_TC_TR_ICE = (np.diff(ICE_TCNN_TC_TR)/np.diff(PD_values_TC_TR))
    beta_TCNN_VOT_TR_ICE = compute_VOT_IND(beta_TCNN_TT_TR_ICE[:,TC_TR_IDX],beta_TCNN_TC_TR_ICE[:,TC_TR_IDX])

    beta_TCNN_TT_SM_ICE = (np.diff(ICE_TCNN_TT_SM)/np.diff(PD_values_TT_SM))
    beta_TCNN_TC_SM_ICE = (np.diff(ICE_TCNN_TC_SM)/np.diff(PD_values_TC_SM))
    beta_TCNN_VOT_SM_ICE = compute_VOT_IND(beta_TCNN_TT_SM_ICE[:,TC_SM_IDX],beta_TCNN_TC_SM_ICE[:,TC_SM_IDX])


    beta_TCNN_TT_CAR_ICE = (np.diff(ICE_TCNN_TT_CAR)/np.diff(PD_values_TT_CAR))
    beta_TCNN_TC_CAR_ICE = (np.diff(ICE_TCNN_TC_CAR)/np.diff(PD_values_TC_CAR))
    beta_TCNN_VOT_CAR_ICE = compute_VOT_IND(beta_TCNN_TT_CAR_ICE[:,TC_CAR_IDX],beta_TCNN_TC_CAR_ICE[:,TC_CAR_IDX])


    # Estimation # If we use the median value, we don't need to care about the outlier or somethings.
  
    x_test_E = x_train[['AGES','INCOMES','PURPOSES']].reset_index(drop=True)
    x_test_E['VOT_TR'] = beta_TCNN_VOT_TR_ICE
    x_test_E['VOT_SM'] = beta_TCNN_VOT_SM_ICE
    x_test_E['VOT_CAR'] = beta_TCNN_VOT_CAR_ICE

    VOT_TCNN_IND = x_test_E[['AGES','INCOMES','PURPOSES','VOT_TR','VOT_SM','VOT_CAR']].groupby(['AGES','INCOMES','PURPOSES']).median()
   
        
    print(np.quantile(VOT_TCNN_IND['VOT_TR'],(0.5,0.01,0.25,0.75,0.99)).round(3))
    print(np.quantile(VOT_TCNN_IND['VOT_SM'],(0.5,0.01,0.25,0.75,0.99)).round(3))
    print(np.quantile(VOT_TCNN_IND['VOT_CAR'],(0.5,0.01,0.25,0.75,0.99)).round(3))
    

    # Estimation results
    PRED_PERF = np.array([train_acc,test_acc,train_brier,test_brier]).transpose()
    EST_RESULTS = [PRED_PERF,VOT_TCNN_IND,training_time]
    
    
    with open("C:/Users/euijin/Documents/LatDNN_Results_Revision/ASC_RESULTS_FF_"+str(i), "wb") as f:
        pickle.dump(EST_RESULTS, f)
    #np.savetxt("C:/Users/euijin/Documents/LatDNN_Results_Revision/ASC_RESULTS_FF_"+str(i),EST_RESULTS)
    #save_clipboard(np.concatenate([PRED_PERF,VOT_TCNN_IND.median(axis=0).round(3)])) 


In [ ]:
import scipy.stats as stats
## Results analysis
PRED_PERF_S = []
VOT_TCNN_IND_S = []
VOT_TCNN_index = []

for i in range(18):
    with open("C:/Users/euijin/Documents/LatDNN_Results_Revision/ASC_RESULTS_FF_"+str(i),'rb') as f:
        temp = pickle.load(f)  
    PRED_PERF_S.append(temp[0])
    VOT_TCNN_IND_S.append(temp[1])
    VOT_TCNN_index.append(temp[1].index.values)
    

## Predictability    
PRED_PERF_S = np.array(PRED_PERF_S)
A = np.concatenate([PRED_PERF_S.mean(axis=0).round(3).reshape(-1,1),PRED_PERF_S.std(axis=0).round(3).reshape(-1,1)],axis=1)


## Interpretability
IND_stats = []
VOT_TCNN_IND_S_TR = []
VOT_TCNN_IND_S_SM = []
VOT_TCNN_IND_S_CAR = []

for i in range(len(VOT_TCNN_IND_S)):
    
    inter_IDX_IND = VOT_TCNN_IND_S[i].index.isin(np.intersect1d(VOT_TCNN_IND_S[i].index,df_VOT_B.index))
        
    TRIND = np.quantile(VOT_TCNN_IND_S[i].iloc[inter_IDX_IND,0],(0.5,0.01,0.25,0.75,0.99)).round(3)
    SMIND = np.quantile(VOT_TCNN_IND_S[i].iloc[inter_IDX_IND,1],(0.5,0.01,0.25,0.75,0.99)).round(3)
    CARIND = np.quantile(VOT_TCNN_IND_S[i].iloc[inter_IDX_IND,2],(0.5,0.01,0.25,0.75,0.99)).round(3)   
    ALLIND = np.concatenate([TRIND,SMIND,CARIND])   
    IND_stats.append(ALLIND)
    
    VOT_TCNN_IND_S_TR.append(VOT_TCNN_IND_S[i].iloc[inter_IDX_IND,0])
    VOT_TCNN_IND_S_SM.append(VOT_TCNN_IND_S[i].iloc[inter_IDX_IND,1])
    VOT_TCNN_IND_S_CAR.append(VOT_TCNN_IND_S[i].iloc[inter_IDX_IND,2])
      
B = np.array(IND_stats)
B = np.concatenate([np.median(B,axis=0).round(3).reshape(-1,1),
                    stats.iqr(B,axis=0).round(3).reshape(-1,1)],axis=1)


VOT_TCNN_IND_S_TR = np.concatenate(VOT_TCNN_IND_S_TR)
VOT_TCNN_IND_S_SM = np.concatenate(VOT_TCNN_IND_S_SM)
VOT_TCNN_IND_S_CAR = np.concatenate(VOT_TCNN_IND_S_CAR)

EST_RESULTS_TCNN = np.vstack([B,A])
save_clipboard(EST_RESULTS_TCNN)

In [ ]:
import scipy.stats as stats

## Results analysis of MNL

PRED_PERF_S = []
PARAM_PERF_S = []
VOT_MNL_IND_S = []
VOT_TRUE_S = []
VOT_MNL_index = []


## Results analysis of TCNN

PRED_PERF_S = []
PARAM_PERF_S = []
VOT_TCNN_IND_S = []
VOT_TRUE_S = []
VOT_TCNN_index = []

for i in range(1):
    with open("C:/Users/euijin/Documents/LatDNN_Results_Revision/ASC_RESULTS_FF_"+str(i),'rb') as f:
        temp = pickle.load(f)  
    PRED_PERF_S.append(temp[0])
    VOT_TCNN_IND_S.append(temp[1])
    VOT_TCNN_index.append(temp[1].index.values)

df_VOT_TCNN = pd.DataFrame(np.vstack(VOT_TCNN_IND_S),index = np.hstack(VOT_TCNN_index),columns = ['VOT_TR','VOT_SM','VOT_CAR'])


## Results analysis of DNN
PRED_PERF_S = []
PARAM_PERF_S = []
VOT_DNN_IND_S = []
VOT_TRUE_S = []
VOT_DNN_index = []

for i in range(1):
    with open("C:/Users/euijin/Documents/DNN_Results_Revision/FUL_RESULTS_FF_"+str(i),'rb') as f:
        temp = pickle.load(f)  
    PRED_PERF_S.append(temp[0])
    VOT_DNN_IND_S.append(temp[1])
    VOT_DNN_index.append(temp[1].index.values)

df_VOT_DNN = pd.DataFrame(np.vstack(VOT_DNN_IND_S),index = np.hstack(VOT_DNN_index),columns = ['VOT_TR','VOT_SM','VOT_CAR'])

## Results analysis of MNL
df_VOT_MNL = pd.DataFrame(df_VOT_B.values[:,1:4],index = df_VOT_B.index.values,columns =['VOT_TR','VOT_SM','VOT_CAR'])